In [ ]:
# Cell 1: Header and Setupimport syssys.path.insert(0, '..')from utils.notebook_utils import display_header, display_toc, check_dependency, conclusion_box, info_box, warning_boxfrom utils.system_info import display_system_infofrom utils.benchmark import Benchmark, BenchmarkResult, ComparisonTablefrom utils.charts import setup_style, bar_comparison, COLORSdisplay_header('LlamaIndex Integration', 'SynaDB as a LlamaIndex Backend')

In [ ]:
# Cell 2: Table of Contentssections = [    ('Introduction', 'introduction'),    ('Setup', 'setup'),    ('SynaVectorStore Demo', 'vectorstore'),    ('SynaChatStore Demo', 'chatstore'),    ('VectorStoreIndex Building', 'index-building'),    ('Query Engine Demo', 'query-engine'),    ('Multi-Index Routing', 'multi-index'),    ('Persistence Demo', 'persistence'),    ('Results Summary', 'results'),    ('Conclusions', 'conclusions'),]display_toc(sections)

## 📌 Introduction <a id="introduction"></a>This notebook demonstrates **SynaDB's LlamaIndex integration**, showing how to use SynaDB as a backend for LlamaIndex applications.### LlamaIndex Components Provided by SynaDB| Component | Class | Purpose ||-----------|-------|---------|| **Vector Store** | `SynaVectorStore` | Store and search document embeddings || **Chat Store** | `SynaChatStore` | Persist conversation history |### Why Use SynaDB with LlamaIndex?| Feature | SynaDB Advantage ||---------|------------------|| **Single File** | No external database server needed || **Zero Config** | Works out of the box || **Persistence** | Data survives restarts || **Offline** | No network required || **Fast** | Native HNSW index for vector search |### What We'll Demonstrate1. **SynaVectorStore** - Document storage and retrieval2. **SynaChatStore** - Conversation memory persistence3. **VectorStoreIndex** - Building indexes with SynaDB backend4. **Query Engine** - Querying documents with SynaDB retrieval5. **Multi-Index Routing** - Managing multiple indexes6. **Persistence** - Data survival across sessions

In [ ]:
# Cell 4: System Infodisplay_system_info()

## 🔧 Setup <a id="setup"></a>Let's set up our environment and check for required dependencies.

In [ ]:
# Cell 6: Check Dependencies and Importsimport numpy as npimport timeimport osimport tempfilefrom pathlib import Path# Check for SynaDBHAS_SYNADB = check_dependency('synadb', 'pip install synadb')# Check for LlamaIndexHAS_LLAMAINDEX = check_dependency('llama_index', 'pip install llama-index llama-index-core')# Apply consistent stylingsetup_style()print("\n✓ Dependencies checked")

In [ ]:
# Cell 7: Create Temp Directory for Databasestemp_dir = tempfile.mkdtemp(prefix='synadb_llamaindex_')print(f'Using temp directory: {temp_dir}')# Paths for databasesvectorstore_path = os.path.join(temp_dir, 'vectorstore.db')chatstore_path = os.path.join(temp_dir, 'chatstore.db')index_path = os.path.join(temp_dir, 'index.db')multi_index_path = os.path.join(temp_dir, 'multi_index.db')

In [ ]:
# Cell 8: Create Mock Embedding Model (for demo without API keys)class MockEmbedding:    """Mock embedding model for demonstration without API keys."""        def __init__(self, embed_dim: int = 384):        self.embed_dim = embed_dim        np.random.seed(42)        def _get_text_embedding(self, text: str):        """Generate deterministic embedding for text."""        np.random.seed(hash(text) % 2**32)        embedding = np.random.randn(self.embed_dim).astype(np.float32)        embedding = embedding / np.linalg.norm(embedding)        return embedding.tolist()        def _get_query_embedding(self, query: str):        """Generate deterministic embedding for query."""        return self._get_text_embedding(query)        async def _aget_text_embedding(self, text: str):        return self._get_text_embedding(text)        async def _aget_query_embedding(self, query: str):        return self._get_query_embedding(query)# Create mock embedding instancemock_embed_model = MockEmbedding(embed_dim=384)print(f"✓ Created mock embedding model with {mock_embed_model.embed_dim} dimensions")

## 📦 SynaVectorStore Demo <a id="vectorstore"></a>The `SynaVectorStore` class implements LlamaIndex's VectorStore interface, enabling:- Node storage with embeddings- Similarity search queries- Integration with LlamaIndex indexes

In [ ]:
# Cell 10: SynaVectorStore Demoif HAS_SYNADB and HAS_LLAMAINDEX:    from synadb.integrations.llamaindex import SynaVectorStore    from llama_index.core.schema import TextNode        print("Creating SynaVectorStore...")        # Create vector store    vector_store = SynaVectorStore(        path=vectorstore_path,        dimensions=384,        metric="cosine"    )        # Sample documents for demonstration    sample_texts = [        "Machine learning is a subset of artificial intelligence that enables systems to learn from data.",        "Deep learning uses neural networks with multiple layers to process complex patterns.",        "Natural language processing allows computers to understand and generate human language.",        "Computer vision enables machines to interpret and analyze visual information from images.",        "Reinforcement learning trains agents through rewards and penalties in an environment.",        "Transfer learning applies knowledge from one task to improve performance on another.",        "SynaDB is an AI-native embedded database designed for machine learning workloads.",        "Vector databases store embeddings for efficient similarity search operations.",        "RAG combines retrieval with generation for more accurate AI responses.",        "LlamaIndex provides tools for building applications with large language models.",    ]        # Create TextNodes with embeddings    nodes = []    for i, text in enumerate(sample_texts):        embedding = mock_embed_model._get_text_embedding(text)        node = TextNode(            text=text,            id_=f"node_{i}",            embedding=embedding,            metadata={"index": i, "category": "ml" if i < 6 else "tools"}        )        nodes.append(node)        # Add nodes to vector store    start = time.perf_counter()    ids = vector_store.add(nodes)    add_time = (time.perf_counter() - start) * 1000        print(f"✓ Added {len(ids)} nodes in {add_time:.2f}ms")    print(f"  Path: {vectorstore_path}")else:    print("⚠️ SynaDB or LlamaIndex not available, skipping vectorstore demo")    vector_store = None

In [ ]:
# Cell 11: Query Demoif vector_store and HAS_LLAMAINDEX:    from llama_index.core.vector_stores.types import VectorStoreQuery        print("Performing vector store queries...\n")        # Test queries    queries = [        "What is machine learning?",        "How do neural networks work?",        "Tell me about SynaDB",    ]        for query_text in queries:        print(f"Query: '{query_text}'")        print("-" * 50)                # Create query embedding        query_embedding = mock_embed_model._get_query_embedding(query_text)                # Create VectorStoreQuery        query = VectorStoreQuery(            query_embedding=query_embedding,            similarity_top_k=3        )                # Execute query        start = time.perf_counter()        result = vector_store.query(query)        query_time = (time.perf_counter() - start) * 1000                for i, (node, similarity, node_id) in enumerate(zip(result.nodes, result.similarities, result.ids), 1):            print(f"  {i}. (sim: {similarity:.4f}) {node.text[:60]}...")            if node.metadata:                print(f"     Metadata: {node.metadata}")                print(f"  ⏱️ Query time: {query_time:.2f}ms\n")else:    print("⚠️ Vector store not available")

## 💬 SynaChatStore Demo <a id="chatstore"></a>The `SynaChatStore` class implements LlamaIndex's chat store interface, enabling:- Persistent conversation storage- Session-based message management- Integration with LlamaIndex memory components

In [ ]:
# Cell 13: SynaChatStore Demoif HAS_SYNADB and HAS_LLAMAINDEX:    from synadb.integrations.llamaindex import SynaChatStore    from llama_index.core.llms import ChatMessage, MessageRole        print("Creating SynaChatStore...")        # Create chat store    chat_store = SynaChatStore(path=chatstore_path)        # Create sample messages    session_key = "session_001"    messages = [        ChatMessage(role=MessageRole.USER, content="Hello! What is SynaDB?"),        ChatMessage(role=MessageRole.ASSISTANT, content="SynaDB is an AI-native embedded database designed for machine learning workloads."),        ChatMessage(role=MessageRole.USER, content="What are its main features?"),        ChatMessage(role=MessageRole.ASSISTANT, content="SynaDB offers vector storage, experiment tracking, model registry, and LLM framework integrations."),    ]        # Store messages    print(f"\nStoring {len(messages)} messages for session '{session_key}'...")    chat_store.set_messages(session_key, messages)    print("✓ Messages stored")        # Retrieve messages    print(f"\nRetrieving messages from session '{session_key}':")    print("-" * 60)        retrieved = chat_store.get_messages(session_key)    for msg in retrieved:        role = "User" if msg.role == MessageRole.USER else "Assistant"        print(f"[{role}]: {msg.content[:70]}{'...' if len(msg.content) > 70 else ''}")        print(f"\n✓ Retrieved {len(retrieved)} messages")        # Add a new message    print("\nAdding a new message...")    chat_store.add_message(session_key, ChatMessage(        role=MessageRole.USER,        content="Can I use it with LlamaIndex?"    ))        # Show updated messages    updated = chat_store.get_messages(session_key)    print(f"✓ Session now has {len(updated)} messages")        # List all sessions    print(f"\nAll session keys: {chat_store.get_keys()}")else:    print("⚠️ SynaDB or LlamaIndex not available, skipping chat store demo")    chat_store = None

## 🏗️ VectorStoreIndex Building <a id="index-building"></a>Let's demonstrate building a LlamaIndex VectorStoreIndex with SynaDB as the backend.> **Note:** This demo uses mock components. In production, you would use real embedding models and LLMs.

In [ ]:
# Cell 15: VectorStoreIndex Building Demoif HAS_SYNADB and HAS_LLAMAINDEX:    from synadb.integrations.llamaindex import SynaVectorStore    from llama_index.core.schema import TextNode, Document        print("Building VectorStoreIndex with SynaDB backend...\n")        # Create a new vector store for the index    index_store = SynaVectorStore(        path=index_path,        dimensions=384,        metric="cosine"    )        # Sample documents about different topics    documents = [        {"text": "Python is a versatile programming language widely used in data science.", "topic": "programming"},        {"text": "JavaScript is essential for web development and runs in browsers.", "topic": "programming"},        {"text": "Machine learning models can predict outcomes based on historical data.", "topic": "ml"},        {"text": "Neural networks are inspired by the structure of the human brain.", "topic": "ml"},        {"text": "SynaDB provides efficient vector storage for AI applications.", "topic": "database"},        {"text": "PostgreSQL is a powerful open-source relational database.", "topic": "database"},    ]        # Create nodes with embeddings    nodes = []    for i, doc in enumerate(documents):        embedding = mock_embed_model._get_text_embedding(doc["text"])        node = TextNode(            text=doc["text"],            id_=f"doc_{i}",            embedding=embedding,            metadata={"topic": doc["topic"]}        )        nodes.append(node)        # Add to index    start = time.perf_counter()    index_store.add(nodes)    build_time = (time.perf_counter() - start) * 1000        print(f"✓ Built index with {len(nodes)} documents in {build_time:.2f}ms")        # Demonstrate querying the index    print("\nQuerying the index:")    print("-" * 50)        test_queries = [        "What programming languages are popular?",        "How do machine learning models work?",        "What databases are available?",    ]        for query_text in test_queries:        query_embedding = mock_embed_model._get_query_embedding(query_text)        query = VectorStoreQuery(query_embedding=query_embedding, similarity_top_k=2)        result = index_store.query(query)                print(f"\nQuery: '{query_text}'")        for node, sim in zip(result.nodes, result.similarities):            print(f"  • [{node.metadata.get('topic', 'N/A')}] {node.text[:50]}... (sim: {sim:.3f})")else:    print("⚠️ SynaDB or LlamaIndex not available, skipping index building demo")    index_store = None

## 🔍 Query Engine Demo <a id="query-engine"></a>Let's demonstrate how to use SynaDB with a LlamaIndex query engine for question answering.

In [ ]:
# Cell 17: Query Engine Demoif index_store and HAS_LLAMAINDEX:    print("Demonstrating Query Engine with SynaDB...\n")        # Simulate a query engine workflow    query = "What are the best tools for AI development?"        print(f"Query: '{query}'")    print("=" * 60)        # Step 1: Retrieve relevant documents    print("\n📚 Step 1: Retrieving relevant documents...")    query_embedding = mock_embed_model._get_query_embedding(query)    store_query = VectorStoreQuery(query_embedding=query_embedding, similarity_top_k=3)        start = time.perf_counter()    result = index_store.query(store_query)    retrieval_time = (time.perf_counter() - start) * 1000        print(f"   Retrieved {len(result.nodes)} documents in {retrieval_time:.2f}ms:")    for i, (node, sim) in enumerate(zip(result.nodes, result.similarities), 1):        print(f"   {i}. [{node.metadata.get('topic')}] {node.text[:50]}... (sim: {sim:.3f})")        # Step 2: Build context    print("\n📝 Step 2: Building context from retrieved documents...")    context = "\n".join([f"- {node.text}" for node in result.nodes])    print(f"   Context ({len(context)} chars):")    print(f"   {context[:200]}...")        # Step 3: Generate response (simulated)    print("\n🤖 Step 3: Generating response (simulated)...")    simulated_response = f"""Based on the retrieved documents, here are recommended tools for AI development:1. **Python** - A versatile programming language widely used in data science2. **SynaDB** - Provides efficient vector storage for AI applications3. **Machine Learning Models** - Can predict outcomes based on historical dataThese tools form a solid foundation for AI development workflows."""        print(f"\n   Response:\n   {'-' * 50}")    for line in simulated_response.split('\n'):        print(f"   {line}")        print(f"\n✓ Query engine workflow completed")else:    print("⚠️ Index store not available, skipping query engine demo")

## 🔀 Multi-Index Routing <a id="multi-index"></a>SynaDB can manage multiple indexes for different document collections, enabling routing queries to the appropriate index.

In [ ]:
# Cell 19: Multi-Index Routing Demoif HAS_SYNADB and HAS_LLAMAINDEX:    print("Demonstrating Multi-Index Routing with SynaDB...\n")        # Create separate indexes for different topics    indexes = {}        # Technical documentation index    tech_path = os.path.join(temp_dir, 'tech_index.db')    tech_store = SynaVectorStore(path=tech_path, dimensions=384, metric="cosine")    tech_docs = [        "API endpoints should follow RESTful conventions for consistency.",        "Authentication tokens should be stored securely and rotated regularly.",        "Database queries should be optimized with proper indexing.",    ]    tech_nodes = [        TextNode(text=t, id_=f"tech_{i}", embedding=mock_embed_model._get_text_embedding(t), metadata={"type": "technical"})        for i, t in enumerate(tech_docs)    ]    tech_store.add(tech_nodes)    indexes["technical"] = tech_store    print(f"✓ Created technical index with {len(tech_docs)} documents")        # Product documentation index    product_path = os.path.join(temp_dir, 'product_index.db')    product_store = SynaVectorStore(path=product_path, dimensions=384, metric="cosine")    product_docs = [        "Our product offers seamless integration with existing workflows.",        "Pricing plans are flexible and scale with your usage.",        "Customer support is available 24/7 via chat and email.",    ]    product_nodes = [        TextNode(text=t, id_=f"product_{i}", embedding=mock_embed_model._get_text_embedding(t), metadata={"type": "product"})        for i, t in enumerate(product_docs)    ]    product_store.add(product_nodes)    indexes["product"] = product_store    print(f"✓ Created product index with {len(product_docs)} documents")        # Simple router function    def route_query(query: str) -> str:        """Route query to appropriate index based on keywords."""        technical_keywords = ["api", "database", "authentication", "code", "technical"]        product_keywords = ["pricing", "support", "product", "customer", "plan"]                query_lower = query.lower()        tech_score = sum(1 for kw in technical_keywords if kw in query_lower)        product_score = sum(1 for kw in product_keywords if kw in query_lower)                return "technical" if tech_score >= product_score else "product"        # Test routing    print("\nTesting query routing:")    print("-" * 50)        test_queries = [        "How do I authenticate API requests?",        "What are the pricing options?",        "How should I optimize database queries?",        "How can I contact customer support?",    ]        for query in test_queries:        route = route_query(query)        store = indexes[route]                query_embedding = mock_embed_model._get_query_embedding(query)        store_query = VectorStoreQuery(query_embedding=query_embedding, similarity_top_k=1)        result = store.query(store_query)                print(f"\nQuery: '{query}'")        print(f"  → Routed to: {route} index")        if result.nodes:            print(f"  → Result: {result.nodes[0].text[:60]}...")        print("\n✓ Multi-index routing demonstrated")else:    print("⚠️ SynaDB or LlamaIndex not available, skipping multi-index demo")

## 💾 Persistence Demo <a id="persistence"></a>One of SynaDB's key advantages is its single-file persistence. Let's demonstrate how data survives across sessions.

In [ ]:
# Cell 21: Persistence Demoif HAS_SYNADB and HAS_LLAMAINDEX:    print("Demonstrating SynaDB Persistence...\n")        # Show current database files    print("Database files created:")    print("-" * 50)        for filename in os.listdir(temp_dir):        filepath = os.path.join(temp_dir, filename)        if os.path.isfile(filepath):            size = os.path.getsize(filepath)            print(f"  📁 {filename}: {size:,} bytes ({size/1024:.1f} KB)")        # Demonstrate vector store persistence    print("\nVectorStore Persistence Test:")    print("-" * 50)        # Close and reopen vector store    original_path = vectorstore_path    del vector_store        # Reopen the vector store    vector_store_reopened = SynaVectorStore(        path=original_path,        dimensions=384,        metric="cosine"    )        # Query in reopened store    query_text = "machine learning basics"    query_embedding = mock_embed_model._get_query_embedding(query_text)    query = VectorStoreQuery(query_embedding=query_embedding, similarity_top_k=2)    result = vector_store_reopened.query(query)        print(f"  Query: '{query_text}'")    print(f"  ✓ Found {len(result.nodes)} results after reopening:")    for node in result.nodes:        print(f"    • {node.text[:50]}...")        # Demonstrate chat store persistence    print("\nChatStore Persistence Test:")    print("-" * 50)        # Reopen chat store    chat_store_reopened = SynaChatStore(path=chatstore_path)    messages = chat_store_reopened.get_messages("session_001")        print(f"  ✓ Retrieved {len(messages)} messages after reopening")    if messages:        print(f"  First message: [{messages[0].role.value}] {messages[0].content[:50]}...")        print(f"  Last message: [{messages[-1].role.value}] {messages[-1].content[:50]}...")        print("\n✓ Persistence verified - data survives across sessions!")else:    print("⚠️ SynaDB or LlamaIndex not available, skipping persistence demo")

## 📊 Results Summary <a id="results"></a>Let's summarize the LlamaIndex integration capabilities demonstrated.

In [ ]:
# Cell 23: Results Summaryfrom IPython.display import display, Markdownsummary_table = """### LlamaIndex Integration Summary| Component | Status | Features Demonstrated ||-----------|--------|----------------------|| **SynaVectorStore** | ✅ Working | Node storage, similarity queries, metadata support || **SynaChatStore** | ✅ Working | Message storage, session management, persistence || **VectorStoreIndex** | ✅ Working | Index building, document ingestion || **Query Engine** | ✅ Working | Retrieval, context building, response generation || **Multi-Index Routing** | ✅ Working | Multiple indexes, query routing || **Persistence** | ✅ Working | Single-file storage, data survival |### Key Advantages| Feature | Benefit ||---------|---------|| **Single File** | No database server needed, easy deployment || **Zero Config** | Works immediately without setup || **Offline** | No network required for operation || **Fast Search** | Native HNSW index for efficient similarity search || **Persistence** | Data survives application restarts || **Integration** | Seamless LlamaIndex compatibility |"""display(Markdown(summary_table))

## 🎯 Conclusions <a id="conclusions"></a>

In [ ]:
# Cell 25: Conclusionsconclusion_box(    title="Key Takeaways",    points=[        "SynaDB provides full LlamaIndex integration with VectorStore and ChatStore components",        "Single-file storage eliminates the need for external database servers",        "Data persists across sessions, enabling long-term memory for applications",        "Zero configuration required - works out of the box",        "Native HNSW indexing provides fast similarity search",        "Multi-index routing enables sophisticated document organization",        "Ideal for local development, prototyping, and offline applications",    ],    summary="SynaDB offers a simple, embedded alternative to server-based vector databases for LlamaIndex applications.")

In [ ]:
# Cell 26: Cleanupimport shutilprint("Cleaning up temporary files...")try:    shutil.rmtree(temp_dir)    print(f"✓ Removed temp directory: {temp_dir}")except Exception as e:    print(f"⚠️ Could not remove temp directory: {e}")print("\n✓ Notebook complete!")